In [1]:
import numpy as np
import pandas as pd
import re
import os
from tqdm import tqdm
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import NormalPredictor, KNNBasic, SVD, SVDpp

In [2]:
from recommender_metrics import RecommenderMetrics
from movie_lens_data import MovieLensData
from evaluator import Evaluator

In [3]:
path = "./data/ml-100k"
movie_lens_data = MovieLensData(
    users_path = os.path.join(path, "u.user"),
    ratings_path = os.path.join(path, "u.data_new0"),
    movies_path = os.path.join(path, "u.item"), 
    genre_path = os.path.join(path, "u.genre") 
    )

evaluation_data = movie_lens_data.read_ratings_data()
movie_data = movie_lens_data.read_movies_data()
popularity_rankings = movie_lens_data.get_popularity_ranks()
ratings = movie_lens_data.get_ratings()

In [4]:
evaluator = Evaluator(evaluation_data, popularity_rankings)
# adding random evaluator
# algo_np = NormalPredictor()
# evaluator.add_algorithm(algo_np, "Random")

Number of full trainset users: 935
Number of full trainset items: 1645
Number of trainset users: 935
Number of trainset items: 1645
Size of testset: 79275
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


# Add item-based collaborative filtering RecSys to evaluator


Using the sim_options, we specify the type of similarity calculation and if the collaborative filtering is user based (in this case, No)

In [5]:
item_KNN = KNNBasic(sim_options = {'name': 'cosine', 'user_based': False})
evaluator.add_algorithm(item_KNN, "Item KNN")

ItemKnn

In [6]:
user_KNN = KNNBasic(sim_options = {'name': 'pearson', 'user_based': True})
evaluator.add_algorithm(user_KNN, "User KNN")

In [7]:
SVD = SVD()
evaluator.add_algorithm(SVD, "SVD")

In [8]:
SVD_plus_plus = SVDpp()
evaluator.add_algorithm(SVD_plus_plus, "SVD++")

In [9]:
# evaluator.evaluate(do_top_n=False)

In [10]:
# Time consuming, uncomment optionally
evaluator.evaluate(do_top_n=True)

Evaluating  Item KNN ...
Evaluating accuracy...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating top-N with leave-one-out...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  User KNN ...
Evaluating accuracy...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating top-N with leave-one-out...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Analyzing coverage, diversity, and novelty...
Comput

# Evaluate topN recommendations

In [11]:
# evaluator.sample_top_n_recs(movie_lens_data, test_subject=85, k=10)

In [12]:
# evaluator.sample_top_n_recs(movie_lens_data, test_subject=314, k=5)

In [13]:
# print(movie_lens_data.get_movie_name(44))
# evaluator.sample_top_n_recs(movie_lens_data, test_subject=44, k=10)

In [14]:
# list_of_recommendations=evaluator.sample_top_n_recs(movie_lens_data, test_subject=1, k=10)

In [15]:
# print(list_of_recommendations)

# Evaluation

In [16]:
import numpy as np

def ndcg_score(y_true, y_pred, k=None):
    # Calculate the Normalized Discounted Cumulative Gain (NDCG) score.
    # Convert inputs to numpy arrays
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    # Ensure y_true and y_pred have the same shape
    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")
    
    # Sort indices of y_pred in descending order
    sorted_indices = np.argsort(y_pred)[::-1]
    
    # Select top-K elements if k is specified
    if k is not None:
        sorted_indices = sorted_indices[:k]
    
    # Calculate DCG
    dcg = np.sum((2 ** y_true[sorted_indices] - 1) / np.log2(np.arange(2, len(sorted_indices) + 2)))
    
    # Sort y_true in descending order and calculate ideal DCG
    ideal_sorted_indices = np.argsort(y_true)[::-1]
    ideal_dcg = np.sum((2 ** y_true[ideal_sorted_indices] - 1) / np.log2(np.arange(2, len(ideal_sorted_indices) + 2)))
    
    # Calculate NDCG
    ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0
    return ndcg

In [17]:
def average_precision(y_true, y_pred, k=None):
    """
    Calculate the Average Precision (AP) score for a single query.
    """
    if k is not None:
        y_true = y_true[:k]
        y_pred = y_pred[:k]
    
    # Initialize variables
    num_hits = 0
    precision_sum = 0.0
    true_positives = 0
    
    # Calculate precision at each relevant document
    for i, (true, pred) in enumerate(zip(y_true, y_pred), 1):
        if true:
            true_positives += 1
            precision = true_positives / i
            precision_sum += precision
    
    # Calculate Average Precision
    if true_positives == 0:
        return 0.0
    else:
        return precision_sum / true_positives

In [18]:

def mean_average_precision(y_true_list, y_pred_list, k=None):
    """
    Calculate the Mean Average Precision (MAP) score for a list of queries.
    
    Parameters:
    y_true_list (list of arrays): List of true relevance scores for each query.
    y_pred_list (list of arrays): List of predicted relevance scores for each query.
    
    Returns:
    float: The Mean Average Precision (MAP) score.
    """
    total_queries = len(y_true_list)
    sum_ap = 0.0
    
    for y_true, y_pred in zip(y_true_list, y_pred_list):
        sum_ap += average_precision(y_true, y_pred, k)
    
    # Calculate MAP
    map_score = sum_ap / total_queries
    return map_score



In [19]:
def mean_rating_deviation(y_true, y_pred, k=None):
    if k is not None:
        y_true = y_true[:k]
        y_pred = y_pred[:k]
    
    # Convert inputs to numpy arrays
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    # Ensure y_true and y_pred have the same shape
    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")
    
    # Calculate Mean Rating
    mrd_score = np.mean(y_pred)-np.mean(y_true)
    return mrd_score

In [20]:
def mean_reciprocal_rank(y_true, y_pred):
    """
    Calculate the Mean Reciprocal Rank (MRR) score.
    
    Parameters:
    y_true (array-like): Array of true relevance scores.
    y_pred (array-like): Array of predicted relevance scores.
    
    Returns:
    float: The Mean Reciprocal Rank (MRR) score.
    """
    reciprocal_ranks = []
    for true, pred in zip(y_true, y_pred):
        # Find the index of the first correct prediction (assuming 1-indexing)
        first_correct_index = next((i for i, r in enumerate(pred, start=1) if r == true), None)
        if first_correct_index is not None:
            reciprocal_ranks.append(1.0 / first_correct_index)
    
    # Calculate MRR
    if reciprocal_ranks:
        return sum(reciprocal_ranks) / len(reciprocal_ranks)
    else:
        return 0.0

In [21]:
def calc_score(algo_dict, userid, y_true, y_pred, k=None):
    ndcg = ndcg_score(y_true, y_pred,k)
    # print("NDCG score:", ndcg)
    ap=average_precision(y_true, y_pred,k)
    # print('Average Precision', ap)
    mrd_score = mean_rating_deviation(y_true, y_pred,k)
    # print("Mean Rating (MR) deviation score:", mrd_score)

    if userid in algo_dict:
        # If the key exists
        algo_dict[userid].append([ndcg, ap, mrd_score])
    else:
        # If the key does not exist
        algo_dict[userid]=([ndcg, ap, mrd_score])
    return algo_dict

In [22]:
file_path = "D:\\Projects\\P_DSc\\Intern-IITP\\NB\\data\\ml-100k\\u.data.test"  # Specify the path to your TSV file
test_data = pd.read_csv(file_path, sep='\t')

order=['userid', 'itemid', 'rating', 'timestamp']
test_data.columns= order

def sort_column_within_group(group):
    group=group.sort_values(by='rating', ascending=False)
    return group
test_data = test_data.groupby('userid').apply(sort_column_within_group)
test_data = test_data.reset_index(drop=True)
print(test_data)

       userid  itemid  rating  timestamp
0           1      15       5  875071608
1           1      93       5  875071484
2           1     253       5  874965970
3           1     223       5  876892918
4           1     216       5  876892701
...       ...     ...     ...        ...
20277     943     419       2  888638920
20278     943      97       2  888639445
20279     943     720       1  888640048
20280     943     941       1  888639725
20281     943     399       1  888639886

[20282 rows x 4 columns]


In [30]:
score_dict={
    'ItemKNN':{},
    'UserKNN':{},
    'SVD':{},
    'SVD++':{},
}

userids = set(test_data['userid'])
# userids1 = set(test_data['userid'])
# intersection = set1 & set2

for userid in tqdm(userids):
    
    list_of_recommendations=evaluator.sample_top_n_recs(movie_lens_data, test_subject=userid, k=10)
    
    test_i_user = test_data[test_data['userid'].isin([userid])].drop(columns=['userid', 'timestamp']).sort_values(by='itemid', ascending=True).reset_index(drop=True)
    # test_i_user.to_csv(f'.//data//test_{userid}.csv', index=False)
    # print(test_i_user)
    print(f'\n{userid}\n')

    for i in range(0,4):
        order=['itemid', 'act', 'est']
        prediction_df=pd.DataFrame(list_of_recommendations[i], columns=order).sort_values(by='itemid', ascending=True).reset_index(drop=True)
        # prediction_df.to_csv(f'.//data//prediction{userid}_{i}.csv', index=False)
        # print(prediction_df)
        # print(prediction_df.to_dict)

        selected_items = prediction_df[prediction_df['itemid'].isin(test_i_user['itemid'])]
        selected_items['act'] = test_i_user.loc[test_i_user['itemid'].isin(selected_items['itemid']), 'rating'].values
        selected_items=selected_items.sort_values(by='est', ascending=False)
        # print(selected_items)
        # selected_items.to_csv(f'.//data//selected{userid}_{i}.csv', index=False)

        k=None
        
        y_true = selected_items['act'].values
        y_pred = selected_items['est'].values
        
        algo=('ItemKNN') if (i==0) else ('UserKNN') if(i==1) else ('SVD') if (i==2) else ('SVD++') if (i==3) else ('algo')
        algo_dict=score_dict[algo]
        algo_dict=calc_score(algo_dict, userid, y_true, y_pred, k)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

1

Computing the cosine similarity matrix...


C:\Users\bhask\AppData\Local\Temp\ipykernel_14928\2973332236.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['act'] = test_i_user.loc[test_i_user['itemid'].isin(selected_items['itemid']), 'rating'].values


Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

2

Computing the cosine similarity matrix...


C:\Users\bhask\AppData\Local\Temp\ipykernel_14928\2973332236.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['act'] = test_i_user.loc[test_i_user['itemid'].isin(selected_items['itemid']), 'rating'].values


Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

3

Computing the cosine similarity matrix...


...

C:\Users\bhask\AppData\Local\Temp\ipykernel_14928\2973332236.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['act'] = test_i_user.loc[test_i_user['itemid'].isin(selected_items['itemid']), 'rating'].values


Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

942

Computing the cosine similarity matrix...


C:\Users\bhask\AppData\Local\Temp\ipykernel_14928\2973332236.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['act'] = test_i_user.loc[test_i_user['itemid'].isin(selected_items['itemid']), 'rating'].values


Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

943



C:\Users\bhask\AppData\Local\Temp\ipykernel_14928\2973332236.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['act'] = test_i_user.loc[test_i_user['itemid'].isin(selected_items['itemid']), 'rating'].values


In [31]:
print(score_dict)
score_df=pd.DataFrame(score_dict)
score_df.to_csv(f'.//data//score.csv', index=False)

{'ItemKNN': {1: [0.913833377284391, 1.0, 0.028218400445717773], 2: [0.7876114186959077, 1.0, 0.03886367782789213], 3: [0.6301432645196707, 1.0, 0.12180420158433947], 4: [0.9953395684454479, 1.0, 0.0348836678265565], 5: [0.9018010021736702, 1.0, -0.07362574691777901], 6: [0.8524277018915342, 1.0, 0.24888468856835644], 7: [0.8894113676066036, 1.0, -0.010781145889887878], 8: [0.7480404189354767, 1.0, 0.2550914197568863], 9: [0.826344856390525, 1.0, -0.00841005379782711], 10: [0.9569993824156149, 1.0, 0.06767615143025374], 11: [0.7812523825054491, 1.0, 0.18636589408011828], 12: [0.8418848732980677, 1.0, 0.0753311736021196], 13: [0.6945061561547415, 1.0, -0.3727009250222242], 14: [0.8879134427005081, 1.0, 0.10929304446981103], 15: [0.7376979243602654, 1.0, 0.14304712740440406], 16: [0.9636954680120143, 1.0, 0.19253761621158905], 17: [1.0, 1.0, 0.16762565683693165], 18: [0.7725906976137341, 1.0, 0.006408388805236864], 19: [0.9661884252703736, 1.0, 0.06188213714761748], 20: [0.673855850085905